# Mushroom Classifer Project

In [3]:
import os

In [4]:
all_images = [x for x in os.listdir('MUSHROOM_images/') if x.endswith('.jpg')]
categories = set([x[:-8] for x in all_images])
category2images = {category : [] for category in categories}
for img in all_images:
    category2images[img[:-8]].append(img)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'MUSHROOM_images/'

In [ ]:
all_images

['cat_01b.jpg',
 'cat_04s.jpg',
 'dog_01b.jpg',
 'dog_06s.jpg',
 'mango_01b.jpg',
 'mango_03s.jpg']

In [ ]:
categories

{'cat', 'dog', 'mango'}

In [ ]:
import requests
import torch
from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO
import numpy as np
from tqdm import tqdm
import pandas as pd
import os

In [ ]:
!pip install --upgrade diffusers[torch]
!pip install transformers


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Athena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Athena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from diffusers import StableUnCLIPImg2ImgPipeline
from transformers import CLIPTextModelWithProjection, CLIPTokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# ──────────────────────────────────────────────────────────────
# 1.  Load unCLIP – vision side only (projection_dim = 1024)   ─
# ──────────────────────────────────────────────────────────────
pipe = StableUnCLIPImg2ImgPipeline.from_pretrained(
    "sd2-community/stable-diffusion-2-1-unclip",
    torch_dtype=torch.float16,
).to(device)

vision_encoder = pipe.image_encoder                       # keep as-is (1024-d)

Loading pipeline components...: 100%|██████████| 9/9 [00:13<00:00,  1.49s/it]
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fai

In [ ]:
# ──────────────────────────────────────────────────────────────
# 2.  Swap in an OpenCLIP ViT-H/14 text branch (also 1024-d)  ─
# ──────────────────────────────────────────────────────────────
openclip_repo = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"     # projection_dim = 1024 :contentReference[oaicite:0]{index=0}
tokenizer = CLIPTokenizer.from_pretrained(openclip_repo)
text_encoder = CLIPTextModelWithProjection.from_pretrained(
    openclip_repo,
    torch_dtype=torch.float16
).to(device)

# optional: stuff them into the pipe so `pipe.tokenizer` etc. work
pipe.tokenizer, pipe.text_encoder = tokenizer, text_encoder

In [ ]:
# ──────────────────────────────────────────────────────────────
# 3. Helpers                                              
# ──────────────────────────────────────────────────────────────
def embed_images(paths, batch_size=8):
    """Return (N,1024) image embeddings"""
    out, fe, enc = [], pipe.feature_extractor, pipe.image_encoder
    for i in range(0, len(paths), batch_size):
        imgs = [Image.open(p).convert("RGB") for p in paths[i:i + batch_size]]
        px   = fe(imgs, return_tensors="pt").pixel_values.to(enc.device, enc.dtype)
        with torch.no_grad():
            v = enc(px)[0]                              # (B,1024)
        out.append(v)
    return torch.cat(out)  # (N,1024)

def embed_texts(prompts, batch_size=64):
    """Return (N,1024) text embeddings"""
    vecs = []
    for i in range(0, len(prompts), batch_size):
        toks = tokenizer(prompts[i:i + batch_size],
                         padding=True, truncation=True, max_length=77,
                         return_tensors="pt").to(text_encoder.device)
        with torch.no_grad():
            t = text_encoder(**toks).text_embeds        # (B,1024)
        vecs.append(t)
    return torch.cat(vecs)  # (N,1024)
    

In [ ]:
os.listdir('THINGS_images/test_images/')

['test1.jpg', 'test2.jpg', 'test3.jpg', 'test4.jpg']

In [ ]:
def classify(pipe, image, label):
    '''
    This function classifies an image using CLIP
    - image: is the image path
    - pipe: is the image encoder + text encoder
    - label: takes in a list of possible labels
    '''

    # labels
    labels = label

    # CLIP image and text
    img = embed_images([image])
    txt = embed_texts(labels)

    #normalize for cosine similarity
    img = torch.nn.functional.normalize(img, dim=-1)
    txt = torch.nn.functional.normalize(txt, dim=-1)

    #compute cosine similarities 
    sim = (img @ txt.T).squeeze(0)

    # pick label
    best = sim.argmax().item()
    best_label = labels[best]
    best_score = sim[best].item()

    return best_label
    #, best_score, sim

In [ ]:
print(classify(pipe, "THINGS_images/mango_01b.jpg", ["cat", "mango"]))

mango


# PCA (all functions)

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# embeddings is the numpy array/matrix of the embedded images
#n_components is how many principal components we want 
def run_pca(embeds, n = 2):
    pca = PCA(n_components = n)
    reduced = pca.fit_transform(embeds)
    return reduced, pca

#plot sorted eigenvalues to see drop off (how many components actually matter)
#make sure to run ^pca with as many prinicipal components as possible before this 
def plot_eigenvalues(pca):
    eigenvalues = pca.explained_variance_

    plt.figure(figsize = (8,4))
    plt.plot(range(1, len(eigenvalues) + 1), eigenvalues, marker = 'o')
    plt.title("PCA eigenvalues")
    plt.xlabel("Principal Component")
    plt.grid(True)
    plt.show()

#requires 2 principal components to make it 2d 
#that's why it's red_embeds (reduced embeddings) should be size (.., 2)
# eg use:::
#   pca2, pca_model = run_pca(embeddings, n_components=2)
#   plot_pca_2d(pca2, labels=species_names)
def plot_pca_2d(red_embeds, labels = None, title = "PCA Scatter Plot"):
    plt.figure(figsize = (6,6))
    if labels is None:
        plt.scatter(red_embeds[:, 0], red_embeds[:,1]) #basically plots all points as one color
    else:
        for lab in np.unique(labels):
            idx = [i for i, x in enumerate(labels) if x == lab]
            plt.scatter(red_embeds[idx, 0], red_embeds[idx, 1], label=str(lab))
        plt.legend()

    plt.title(title)
    plt.grid(True)
    plt.show()
    

# K-means (all funtions)


In [9]:
# NOTE: we can use regular image embeddings or PCA reduced
# note: this imports KMeans, but we can manually do it as well
from sklearn.cluster import KMeans

def cluster(img_embeds, text_embeds = None, n = 2, rs = 42):
    """
    Params: 
    img_embeds is the numpy array of image embeddings 
    text_embeds is the numpy array of text embeddings (can b omitted for just image kmeans)
    n is number of clusters (how many diff mushrooms we put in)
    rs is reproducibility ig but idk what that means 

    returns cluster labels( as an array of cluster indices)
    and kmeans 

    """

    if text_embeds is not None:
        X = np.concatenate([img_embeds, text_embeds], axis = 1)
    else:
        X = img_embeds

    kmeans = KMeans(n_clusters = n, random_state = rs)
    kmeans.fit(X)

    cluster_labels = kmeans.labels_
    return cluster_labels, kmeans

#ok to plot and see the k-means, run plot_pca_2d with a 2d pca and labels=clusters, where clusters is the cluster labels we get from k-means


# Classify tests

In [ ]:
# classifying using text descriptions 
description_labels = ["small brown mushroom with gills", "red mushroom with white spots"]
predictions_desc = [classify(pipe, img, labels=description_labels) for img in all_images]

# classifying using mushroom names 
species_labels = ["placeholder"]
predictions_names = [classify(pipe, img, labels=species_labels) for img in all_images]